# Covid-19 day to day statistics

## Table of Contents

1. [Introduction](#introduction)<br>
2. [Cleaning Data](#cleaning)<br>
4. [Selecting Data](#selection)<br>
6. [Grouping Data](#grouping)<br>
7. [Visualising Data](#visualise)<br>

<a id="introduction"></a>
## 1. Introduction

The goal of this jupyter notebook is to create a dynamic map to visualize the spreding of the coronavirus around the world day to day. 

### Data

The data for this example is from the [Our world in data](https://covid.ourworldindata.org/data/owid-covid-data.csv) website, that contains a substantial amount of useful information on data and other aspects of the COVID-19 pandemic. The data is maintained by Hannah Ritchie, and was downloaded directly from [Coronavirus Source Data](https://ourworldindata.org/coronavirus-source-data).

We will focus on the information of the columns total_cases and total_ deahts, separated by countries and for every single day since 12/31/2019.

The data source is updated daily, therefore every time you run this complete notebook you obtain the up to date information.

**Let's start with loading the required Python packages and loading our data into the notebook.**

* To run the code, select the below cell by clicking on it, and then click on the `Run` button at the top of the notebook (or use `Shift+Enter`), to run the cells in the notebook
* The numbers in front of the cells tell you in which order you have run them, for instance `[1]`
* When you see a `[*]` the cell is currently running and `[]` means you have not run the cell yet. Make sure run all of them!

In [2]:
import branca.colormap as cm
import folium
import geopandas as gpd
import urllib.request
import numpy as np
import pandas as pd
from folium.plugins import TimeSliderChoropleth

**Read data from a CSV file using the `read_csv` function.**

In [3]:
df = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv',encoding = 'unicode_escape')

**Let's take a first look at the data loaded into the notebook**

* With `df.head()` or `df.tail()` you can view the first five or last five lines from the data  
* Add a number between the brackets `()` to specify the number of lines you want to display., e.g. `df.head(2)`
* Use `df.dtypes` to check the different variables and their datatype
* `df.columns` gives a list of all column names
* `len(df)` gives the number of rows
* `df.shape` gives the number of rows and columns

> **Tip**: to add more cells to run additional commands, activate a cell by clicking on it and then click on the '+' button at the top of the notebook. This will add a new cell. Click on the buttons with the upwards and downwards arrows to move the cells up and down to change their order

In [4]:
df.head(10)

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
0,AFG,Asia,Afghanistan,2019-12-31,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
1,AFG,Asia,Afghanistan,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
2,AFG,Asia,Afghanistan,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
3,AFG,Asia,Afghanistan,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
4,AFG,Asia,Afghanistan,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
5,AFG,Asia,Afghanistan,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
6,AFG,Asia,Afghanistan,2020-01-06,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
7,AFG,Asia,Afghanistan,2020-01-07,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
8,AFG,Asia,Afghanistan,2020-01-08,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
9,AFG,Asia,Afghanistan,2020-01-09,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83


In [5]:
df.columns

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'total_deaths', 'new_deaths', 'total_cases_per_million',
       'new_cases_per_million', 'total_deaths_per_million',
       'new_deaths_per_million', 'total_tests', 'new_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand', 'tests_units',
       'stringency_index', 'population', 'population_density', 'median_age',
       'aged_65_older', 'aged_70_older', 'gdp_per_capita', 'extreme_poverty',
       'cvd_death_rate', 'diabetes_prevalence', 'female_smokers',
       'male_smokers', 'handwashing_facilities', 'hospital_beds_per_thousand',
       'life_expectancy'],
      dtype='object')

In [6]:
len(df)

25497

In [7]:
df.shape

(25497, 34)

<a id="cleaning"></a>
## 2. Cleaning Data

When exploring data there are always transformations needed to get it in the format you need for your analysis, visualisations or models. Below are only a few examples of the endless possibilities. The best way to learn is to find a dataset and try to answer questions with the data.

We want to represent a map of the total cases. But instead of using the absolute values, we will be using the per_million ones, so we can easily compare how spread is the virus among the population of every country. Therefore we can drop the other columns.

In [9]:
countries = df.copy()[['location','date','total_cases_per_million']]

### Adding and deleting rows or columns

Adding a column can be done by creating a new column `new`, which can be dropped using the `drop` function.
For example, let's drop the rows containing the 'World' information, as we can easily calculate it if we need it.

In [10]:
countries[countries['location']== 'World']

,location,date,total_cases_per_million
25258,World,2019-12-31,0.003
25259,World,2020-01-01,0.003
25260,World,2020-01-02,0.003
25261,World,2020-01-03,0.006
25262,World,2020-01-04,0.006
...,...,...,...
25428,World,2020-06-18,1067.243
25429,World,2020-06-19,1085.036
25430,World,2020-06-20,1107.776
25431,World,2020-06-21,1128.552


In [11]:
countries.drop(countries[countries.location == 'World'].index, inplace=True)

### Further Data Cleaning

**Things to check:**

* Is the data tidy: each variable forms a column, each observation forms a row and  each type of observational unit forms a table.
* Are all columns in the right data format?
* Are there missing values?
* Are there unrealistic outliers?

Get a quick overview of the numeric data using the `.describe()` function. If any of the numeric columns are missing this is a probably because of a wrong data type.

In [12]:
countries.describe()

,total_cases_per_million
count,25035.000000
mean,744.498288
std,1964.667213
min,0.000000
25%,2.611000
50%,71.491000
75%,488.398000
max,30325.297000


Checking the max value, we observe float values for the number of total cases, when obviously we can't have a half person. Therefore we cast all the numbers as integers.

In [13]:
countries[['total_cases_per_million']] = countries[['total_cases_per_million']].fillna(0.0).apply(np.ceil).astype(int)
countries.describe()

,total_cases_per_million
count,25322.000000
mean,736.516270
std,1955.107007
min,0.000000
25%,3.000000
50%,68.000000
75%,471.000000
max,30326.000000


For the total cases, we are going to calculate the log of the values, as there is a big diference in magnitude between countries. The maximum is 30000, while 75% are below 435 and the half below 64.

In [14]:
countries.drop(countries[countries.total_cases_per_million <= 0].index, inplace=True)

In [17]:
countries['log_total_cases'] = np.log10(countries['total_cases_per_million']).astype(int)
countries

,location,date,total_cases_per_million,log_total_cases
56,Afghanistan,2020-02-25,1,0
57,Afghanistan,2020-02-26,1,0
58,Afghanistan,2020-02-27,1,0
59,Afghanistan,2020-02-28,1,0
60,Afghanistan,2020-02-29,1,0
...,...,...,...,...
25253,Zimbabwe,2020-06-18,27,1
25254,Zimbabwe,2020-06-19,32,1
25255,Zimbabwe,2020-06-20,33,1
25256,Zimbabwe,2020-06-21,33,1


<a id="visualise"></a>
## 7. Visualising Data

Folium is a powerful Python library that helps you create several types of Leaflet maps. The fact that the Folium results are interactive makes this library very useful for dashboard building.


Specifically, we will use a `Choropleth` map of the countries. A choropleth map is a thematic map in which areas are shaded or patterned in proportion to the measurement of the statistical variable being displayed on the map, such as population density or per-capita income. The choropleth map provides an easy way to visualize how a measurement varies across a geographic area or it shows the level of variability within a region.

In order to create a Choropleth map, we need a GeoJSON file that defines the areas/boundaries of the state, county, or country that we are interested in. In our case, since we are endeavoring to create a world map, we want a GeoJSON that defines the boundaries of all world countries.


In [18]:
# download countries geojson file
url = 'https://opendata.arcgis.com/datasets/a21fdb46d23e4ef896f31475217cbb08_1.geojson'
filename = 'world_countries.geojson'
urllib.request.urlretrieve(url, filename)

('world_countries.geojson', <http.client.HTTPMessage at 0x1c92f990048>)

In [19]:
df_shapes = gpd.read_file('world_countries.geojson')

In [20]:
df_shapes.head()

,OBJECTID,CNTRY_NAME,geometry
0,1,Aruba,"POLYGON ((-69.88223 12.41111, -69.94695 12.436..."
1,2,Antigua and Barbuda,"MULTIPOLYGON (((-61.73889 17.54055, -61.75195 ..."
2,3,Afghanistan,"POLYGON ((61.27656 35.60725, 61.29638 35.62853..."
3,4,Algeria,"POLYGON ((-5.15213 30.18047, -5.13917 30.19236..."
4,5,Azerbaijan,"MULTIPOLYGON (((45.02583 41.03055, 45.00999 41..."


To be able to join both dataframes some preprocessing is required. As we will be joining them by the country name, we have to make sure that the countries are named equally in both of them. This is a manual process. 

In [21]:
df_shapes = df_shapes.replace({'CNTRY_NAME' : 'Bahamas, The'}, 
                                'Bahamas')
df_shapes = df_shapes.replace({'CNTRY_NAME' : 'Byelarus'}, 
                                'Belarus')
df_shapes = df_shapes.replace({'CNTRY_NAME' : 'Ivory Coast'}, 
                                'Cote d\'Ivoire')
df_shapes = df_shapes.replace({'CNTRY_NAME' : 'Faroe Islands'}, 
                                'Faeroe Islands')
df_shapes = df_shapes.replace({'CNTRY_NAME' : 'Falkland Islands (Islas Malvinas)'}, 
                                'Falkland Islands')
df_shapes = df_shapes.replace({'CNTRY_NAME' : 'Gambia, The'}, 
                                'Gambia')
df_shapes = df_shapes.replace({'CNTRY_NAME' : 'Man, Isle of'}, 
                                'Isle of Man')
df_shapes = df_shapes.replace({'CNTRY_NAME' : 'Myanmar (Burma)'}, 
                                'Myanmar')
df_shapes = df_shapes.replace({'CNTRY_NAME' : 'St. Vincent and the Grenadines'}, 
                                'Saint Vincent and the Grenadines')
df_shapes = df_shapes.replace({'CNTRY_NAME' : 'St. Kitts and Nevis'}, 
                                'Saint Kitts and Nevis')
df_shapes = df_shapes.replace({'CNTRY_NAME' : 'St. Lucia'}, 
                                'Saint Lucia')
df_shapes = df_shapes.replace({'CNTRY_NAME' : 'Tanzania, United Republic of'}, 
                                'Tanzania')
df_shapes = df_shapes.replace({'CNTRY_NAME' : 'Zaire'}, 
                                'Democratic Republic of Congo')

In [22]:
df_shapes.drop(columns=['OBJECTID'], inplace=True)
df_shapes = df_shapes.rename(columns={'CNTRY_NAME': 'location'})
joined_df = countries.merge(df_shapes, on='location')
joined_df.head()

,location,date,total_cases_per_million,log_total_cases,geometry
0,Afghanistan,2020-02-25,1,0,"POLYGON ((61.27656 35.60725, 61.29638 35.62853..."
1,Afghanistan,2020-02-26,1,0,"POLYGON ((61.27656 35.60725, 61.29638 35.62853..."
2,Afghanistan,2020-02-27,1,0,"POLYGON ((61.27656 35.60725, 61.29638 35.62853..."
3,Afghanistan,2020-02-28,1,0,"POLYGON ((61.27656 35.60725, 61.29638 35.62853..."
4,Afghanistan,2020-02-29,1,0,"POLYGON ((61.27656 35.60725, 61.29638 35.62853..."


In [26]:
joined_df.date = joined_df.date.values.astype("datetime64[s]").astype(int)
print(joined_df.date)

0        1582588800
1        1582675200
2        1582761600
3        1582848000
4        1582934400
            ...    
21227    1592438400
21228    1592524800
21229    1592611200
21230    1592697600
21231    1592784000
Name: date, Length: 21232, dtype: int32


### Total cases

Now we define a colour map in terms of the total_cases_ID.

In [27]:
max_colour = max(joined_df['log_total_cases'])
min_colour = min(joined_df['log_total_cases'])
cmap = cm.linear.YlOrRd_09.scale(min_colour, max_colour)
joined_df['colour_tc'] = joined_df['log_total_cases'].map(cmap)

In [28]:
country_list = joined_df['location'].unique().tolist()
country_idx = range(len(country_list))

style_dict = {}
for i in country_idx:
    country = country_list[i]
    result = joined_df[joined_df['location'] == country]
    inner_dict = {}
    for _, r in result.iterrows():
        inner_dict[r['date']] = {'color': r['colour_tc'], 'opacity': 0.7}
    style_dict[str(i)] = inner_dict

In [29]:
countries_df = joined_df[['geometry']]
countries_gdf = gpd.GeoDataFrame(countries_df)
countries_gdf = countries_gdf.drop_duplicates().reset_index()

In [31]:
slider_map = folium.Map(min_zoom=2, max_bounds=True)

_ = TimeSliderChoropleth(
    data=countries_gdf.to_json(),
    styledict=style_dict,

).add_to(slider_map)

_ = cmap.add_to(slider_map)
cmap.caption = "Log10 of the total number of cases per million"
slider_map.save(outfile='Covid_total_cases.html')